# gen_model

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Create a simple model.
#inputs = keras.Input(shape=(32,), name="the_input_xxyyz")
inputs = keras.Input(shape=(32,))
#outputs = keras.layers.Dense(1, name="the_output_blarg")(inputs)
outputs = keras.layers.Dense(1)(inputs)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam", loss="mean_squared_error")

# Train the model.
np.random.seed(0)
test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target)

# Calling `save('my_model')` creates a SavedModel folder `my_model`.
# this overwrites any existing model.  The random.seed(0) still doesn't
# create deterministic models, presumably there's enough variation in
# the initial weights to make each one unique.  Bless.
model.save("my_model.keras")

print( model.predict(test_input) )

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2093  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
[[ 0.14289781]
 [ 0.2379354 ]
 [ 0.35827386]
 [ 0.4321771 ]
 [-0.16267292]
 [ 0.6845731 ]
 [ 0.41606614]
 [ 0.6168652 ]
 [ 0.31931493]
 [ 0.4469811 ]
 [ 0.41002825]
 [ 0.7485537 ]
 [-0.11326866]
 [ 0.473927  ]
 [ 0.310815  ]
 [ 0.2844993 ]
 [ 0.6608665 ]
 [ 0.38076952]
 [ 0.01402762]
 [ 0.35470358]
 [ 0.41576076]
 [ 0.54375786]
 [-0.1314947 ]
 [-0.37747133]
 [ 0.6163339 ]
 [ 0.93825763]
 [ 0.40821752]
 [ 0.66381735]
 [ 0.5979548 ]
 [ 0.13388917]
 [ 0.63985676]
 [ 0.55258226]
 [ 0.16528602]
 [ 0.5345301 ]
 [ 0.32652426]
 [ 0.58203095]
 [ 0.29053634]
 [ 0.87580186]
 [ 0.6265462 ]
 [ 0.577502  ]
 [ 0.26674715]
 [ 0.76412016]
 [ 0.7858373 ]
 [ 0.10714486]
 [ 0.400601  ]
 [-0.13569868]
 [ 0.3803907 ]
 [ 0.29877055]
 [ 0.01468262]
 [-0.09873562]
 [ 0.62180895]
 [ 0.83923227]
 [ 0.19216886]
 [ 0.71573395]
 [-0.34124237]
 [ 0.528531  ]
 [ 0.7001429 ]
 [ 0.24565473]
 [ 0.60066885]
 [ 0.16745824]
 [-0.18

In [ ]:
%%writefile load_model_f.F90
program test_program
use TF_Types
use TF_Interface
use iso_c_binding
implicit none

type(TF_Session) :: session
type(TF_SessionOptions) :: sessionoptions
type(TF_Graph) :: graph
type(TF_Status) :: stat
type(TF_Output), dimension(1) :: input_tfoutput, output_tfoutput
character(100) :: vers
character(100), dimension(1) :: tags
type(TF_Tensor), dimension(1) :: input_tensors, output_tensors, test_tensor
type(TF_Operation), dimension(1) :: target_opers

real, dimension(32), target :: raw_data
real, dimension(:), pointer :: output_data_ptr
integer(kind=c_int64_t), dimension(2) :: input_dims
integer(kind=c_int64_t), dimension(2) :: output_dims
type(c_ptr) :: raw_data_ptr
type(c_ptr) :: output_c_data_ptr

raw_data = (/ &
        0.71332126, 0.81275973, 0.66596436, 0.79570779, 0.83973302, 0.76604397, &
        0.84371391, 0.92582056, 0.32038017, 0.0732005, 0.80589203, 0.75226581, &
        0.81602784, 0.59698078, 0.32991729, 0.43125108, 0.4368422, 0.88550326, &
        0.7131253, 0.14951148, 0.22084413, 0.70801317, 0.69433906, 0.62496564, &
        0.50744999, 0.94047845, 0.18191579, 0.2599102, 0.53161889, 0.57402205, &
        0.50751284, 0.65207096 &
        /)


input_dims = (/ 1, 32 /)
output_dims = (/ 1, 1 /)
tags(1) = 'serve'

! Print TensorFlow library version
call TF_Version(vers)
write(*,*)'Tensorflow version', vers

sessionoptions = TF_NewSessionOptions()
graph = TF_NewGraph()
stat = TF_NewStatus()

! Load session (also populates graph)
session = TF_LoadSessionFromSavedModel(sessionoptions, 'my_model', tags, 1, &
    graph, stat)

if (TF_GetCode( stat ) .ne. TF_OK) then
    call TF_Message( stat, vers )
    write(*,*)'woops', TF_GetCode( stat ), vers
    call abort
endif

call TF_DeleteSessionOptions(sessionoptions)

input_tfoutput(1)%oper = TF_GraphOperationByName( graph, "serving_default_input_1" )
input_tfoutput(1)%index = 0
if (.not.c_associated(input_tfoutput(1)%oper%p)) then
    write(*,*)'input not associated'
    stop
endif

output_tfoutput(1)%oper = TF_GraphOperationByName( graph, "StatefulPartitionedCall" )
output_tfoutput(1)%index = 0
if (.not.c_associated(output_tfoutput(1)%oper%p)) then
    write(*,*)'output not associated'
    stop
endif

! Bind the input tensor
raw_data_ptr = c_loc(raw_data)
input_tensors(1) = TF_NewTensor( TF_FLOAT, input_dims, 2, raw_data_ptr, int(128, kind=c_size_t) )

! Run inference
call TF_SessionRun( session, input_tfoutput, input_tensors, 1, output_tfoutput, output_tensors, 1, &
    target_opers, 0, stat )
if (TF_GetCode( stat ) .ne. TF_OK) then
    call TF_Message( stat, vers )
    write(*,*) TF_GetCode( stat ), vers
    call abort
endif

! Bind output tensor
call c_f_pointer( TF_TensorData( output_tensors(1)), output_data_ptr, shape(output_data_ptr) )
write(*,*)'output data', output_data_ptr(1)

if ((output_data_ptr(1) - -0.479371) .gt. 1e-6) then
    write(*,*)'Output does not match, FAILED!'
else
    write(*,*)'Output is correct, SUCCESS!'
endif


! Clean up
call TF_DeleteTensor( input_tensors(1) )
call TF_DeleteTensor( output_tensors(1) )
call TF_DeleteGraph( graph )
call TF_DeleteSession( session, stat )
call TF_DeleteStatus( stat )

end program test_program

O exemplo load_model_f.F90 é compilado junto com a biblioteca. Os paths são hard coded, isso complica mover os arquivos.

Para rodar, configura a linha:

        session = TF_LoadSessionFromSavedModel(sessionoptions, 'my_model', tags, 1, &
                  graph, stat)

Entra no diretório fortran-tf-lib/build e compila com:

        cmake .. -DCMAKE_BUILD_TYPE=Release -DTENSORFLOW_LOCATION=/usr/local/lib
        make

Os arquivos do TF estão no diretório my_model. Saída:

```bash
x@desk:~/.../tf-lib$ ./test_load_model_f
 Tensorflow version2.18.1                                                                                      
2025-04-04 10:47:19.878195: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: my_model
2025-04-04 10:47:19.880432: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-04-04 10:47:19.880456: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: my_model
2025-04-04 10:47:19.881052: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1743774439.897694    7880 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2025-04-04 10:47:19.898439: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-04-04 10:47:19.933083: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: my_model
2025-04-04 10:47:19.938966: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 60809 microseconds.
 output data -0.479371130    
 Output is correct, SUCCESS!
```